In [1]:
%cd practicas/pr404

/media/notebooks/practicas/pr404


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
!hdfs dfs -mkdir /pr404

In [ ]:
!hdfs dfs -put ./countries_gdp_hist.csv /pr404

# Ejercicio 1: Limpieza y Transformacion

In [9]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(";")
    name = line[4]
    year = int(line[6])
    total = float(line[7])

    if year >= 2000 and total > 0:
        print(f"{name}\t{year}\t{total}")


Overwriting mapper1.py


In [23]:
%%writefile reducer1.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    print(line)

Overwriting reducer1.py


In [24]:
!hdfs dfs -rm -r /pr404/salida1

Deleted /pr404/salida1


In [25]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper1.py \
-file reducer1.py \
-mapper mapper1.py \
-reducer reducer1.py \
-input /pr404/countries_gdp_hist.csv \
-output /pr404/salida1

2026-01-08 09:14:05,368 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper1.py, reducer1.py, /tmp/hadoop-unjar6666179775614688032/] [] /tmp/streamjob3959459563989468885.jar tmpDir=null
2026-01-08 09:14:05,865 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2026-01-08 09:14:05,979 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2026-01-08 09:14:06,167 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1767861571157_0003
2026-01-08 09:14:06,508 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-08 09:14:06,588 INFO mapreduce.JobSubmitter: number of splits:2
2026-01-08 09:14:06,672 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1767861571157_0003
2026-01-08 09:14:06,672 INFO mapreduce.JobSubmitter: Executing 

In [26]:
!hdfs dfs -cat /pr404/salida1/part-00000

AFGHANISTAN	2023	17233051620.1117
	
AFGHANISTAN	2000	3521418059.92345
	
AFGHANISTAN	2001	2813571753.87253
	
AFGHANISTAN	2002	3825701438.99963
	
AFGHANISTAN	2003	4520946818.54581
	
AFGHANISTAN	2004	5224896718.67782
	
AFGHANISTAN	2005	6203256538.70967
	
AFGHANISTAN	2006	6971758282.29351
	
AFGHANISTAN	2007	9747886187.39393
	
AFGHANISTAN	2008	10109297047.5432
	
AFGHANISTAN	2009	12416152732.0567
	
AFGHANISTAN	2010	15856668555.8336
	
AFGHANISTAN	2011	17805098206.3141
	
AFGHANISTAN	2012	19907329777.5872
	
AFGHANISTAN	2013	20146416757.5987
	
AFGHANISTAN	2014	20497128555.6972
	
AFGHANISTAN	2015	19134221644.7325
	
AFGHANISTAN	2016	18116572395.0772
	
AFGHANISTAN	2017	18753456497.8159
	
AFGHANISTAN	2018	18053222687.4126
	
AFGHANISTAN	2019	18799444490.1128
	
AFGHANISTAN	2020	19955929052.1496
	
AFGHANISTAN	2021	14259995441.0759
	
AFGHANISTAN	2022	14497243872.1337
	
ALBANIA	2000	3480355258.04122
	
ALBANIA	2023	23547179830.4413
	
ALBANIA	2022	19017242585.7803
	
ALBANIA	2021	18032010563.7558
	
ALBANIA	

In [27]:
!cat countries_gdp_hist.csv | python3 mapper1.py

ARUBA	2000	1873452513.96648
ARUBA	2001	1896456983.24022
ARUBA	2002	1961843575.41899
ARUBA	2003	2044111731.84358
ARUBA	2004	2254830726.25698
ARUBA	2005	2360017318.43575
ARUBA	2006	2469782681.56425
ARUBA	2007	2677641340.78212
ARUBA	2008	2843024581.00559
ARUBA	2009	2553793296.08939
ARUBA	2010	2453597206.70391
ARUBA	2011	2637859217.87709
ARUBA	2012	2615208379.88827
ARUBA	2013	2727849720.67039
ARUBA	2014	2790849720.67039
ARUBA	2015	2962907262.56983
ARUBA	2016	2983635195.53073
ARUBA	2017	3092429050.27933
ARUBA	2018	3276184357.5419
ARUBA	2019	3395798882.68156
ARUBA	2020	2481857122.8401
ARUBA	2021	2929446578.25104
ARUBA	2022	3279343543.60416
ARUBA	2023	3648573136.15155
AFGHANISTAN	2000	3521418059.92345
AFGHANISTAN	2001	2813571753.87253
AFGHANISTAN	2002	3825701438.99963
AFGHANISTAN	2003	4520946818.54581
AFGHANISTAN	2004	5224896718.67782
AFGHANISTAN	2005	6203256538.70967
AFGHANISTAN	2006	6971758282.29351
AFGHANISTAN	2007	9747886187.39393
AFGHANISTAN	2008	10109297047.5432
AFGHANISTAN	2009	1241615

# Ejercicio 2: Agregacion por clave

In [53]:
%%writefile mapper2.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(";")

    region = line[1]
    total = line[7]

    print(f"{region}\t{total}")

Overwriting mapper2.py


In [66]:
%%writefile reducer2.py
#!/usr/bin/env python3

import sys

actual = None
cont = 0
total = 0

for line in sys.stdin:
    line = line.strip().split()

    region = line[0]
    gdp = float(line[1])

    if actual == None:
        actual = region

    if region != actual:
        prom = total / cont

        print(f"{actual}: {prom}")
        actual = region
        total = gdp
        cont = 1

    if actual == region:
        total = total + gdp
        cont = cont + 1

if region is not None:
    prom = total / cont
    print(f"{actual}: {prom}")

Overwriting reducer2.py


In [67]:
!hdfs dfs -rm -r /pr404/salida2

Deleted /pr404/salida2


In [68]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper2.py \
-file reducer2.py \
-mapper mapper2.py \
-reducer reducer2.py \
-input /pr404/countries_gdp_hist.csv \
-output /pr404/salida2

2026-01-08 09:46:08,556 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper2.py, reducer2.py, /tmp/hadoop-unjar192253693588777352/] [] /tmp/streamjob4346209811921907286.jar tmpDir=null
2026-01-08 09:46:09,063 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2026-01-08 09:46:09,181 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2026-01-08 09:46:09,347 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1767861571157_0012
2026-01-08 09:46:09,627 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-08 09:46:09,690 INFO mapreduce.JobSubmitter: number of splits:2
2026-01-08 09:46:09,782 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1767861571157_0012
2026-01-08 09:46:09,782 INFO mapreduce.JobSubmitter: Executing w

In [69]:
!hdfs dfs -cat /pr404/salida2/part-00000

AFRICA: 17415301365.06409	
AMERICAS: 250035838103.24847	
ASIA: 205913399495.55386	
EUROPE: 213130722585.5805	
OCEANIA: 32507030324.41328	


# Ejercicio 3: Maximos por grupo 

In [71]:
%%writefile mapper3.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(";")

    year = line[6]
    name = line[4]
    variation = line[9]

    print(f"{name}\t{year},{variation}")

Overwriting mapper3.py


In [86]:
%%writefile reducer3.py
#!/usr/bin/env python3

import sys

actual = None
mayorv = 0
año = 0

for line in sys.stdin:
    line = line.strip().split()

    pais = line[0]
    yv = line[1]

    if actual == None:
        actual = pais

    if pais == actual:
        yv = yv.split(",")
        y = yv[0]
        v = float(yv[1])

        if v >= mayorv:
            mayorv = v
            año = y

    elif pais != actual:
        print(f"{actual} {año} ({mayorv})")
        actual = pais
        mayorv = 0
        año = 0

if actual is not None:
    print(f"{actual} {año} ({mayorv})")

Overwriting reducer3.py


In [87]:
!hdfs dfs -rm -r /pr404/salida3

Deleted /pr404/salida3


In [88]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper3.py \
-file reducer3.py \
-mapper mapper3.py \
-reducer reducer3.py \
-input /pr404/countries_gdp_hist.csv \
-output /pr404/salida3

2026-01-08 10:08:41,910 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper3.py, reducer3.py, /tmp/hadoop-unjar7119732635779186151/] [] /tmp/streamjob166157514794914838.jar tmpDir=null
2026-01-08 10:08:42,531 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2026-01-08 10:08:42,656 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2026-01-08 10:08:42,812 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1767861571157_0017
2026-01-08 10:08:43,152 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-08 10:08:43,252 INFO mapreduce.JobSubmitter: number of splits:2
2026-01-08 10:08:43,354 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1767861571157_0017
2026-01-08 10:08:43,355 INFO mapreduce.JobSubmitter: Executing w